# airline project

### import data from database and bookings list 

In [19]:
import sqlite3
import pandas as pd
import numpy as np


conn = sqlite3.connect('airline_seating.db') # create a "connection"

c = conn.cursor() # create a "cursor" 
#clean database to the original 
c.execute("UPDATE seating SET name = ' '")
conn.commit()

c.execute("UPDATE metrics SET passengers_refused = ? , passengers_separated = ? ;",(0,0))
conn.commit() # this is a Python command


aircraft_df = pd.read_sql_query("SELECT * FROM seating;", conn) # give the table name and the connection
aircraft_df # inspect

aircraft_df.head()


# conn = sqlite3.connect('airline_seating.db') # create a "connection" to a DB file in the current directory
# c = conn.cursor() # create a "cursor" 
resoverall = pd.DataFrame(c.execute("SELECT DISTINCT seat FROM seating;").fetchall()) # execute a SQL command
columns = resoverall[0].values.tolist()

#SEAT = c.execute("SELECT DISTINCT seat FROM seating;").fetchall() # execute a SQL command
rows = pd.DataFrame(c.execute("SELECT DISTINCT row FROM seating;").fetchall()) # execute a SQL command
index = rows[0].values.tolist()

data = np.zeros((len(index), len(columns)))
aircraft = pd.DataFrame(data, index=index, columns=columns)
aircraft
#use to get the seat letter after set passager in to airplane
seat_letter = aircraft.columns.values.tolist()
seat_letter[0]

seats_number = len(index)*len(columns)


In [20]:
bookings = pd.read_csv('bookings.csv', header = None)

#bookings.head()
total_bookings_number = len(bookings)


In [25]:
bookings

,0,1
0,Janis Mccullough,3
1,Camille Salinas,1
2,Maria Bonine,4
3,Jesse Stacy,2
4,Louis Cozzolino,3
5,Carmen Burch,1
6,Johnny Mcintyre,1
7,Paul Zehender,1
8,Roy Scott,2
9,Doris Banks,3


In [21]:
#aircraft_left_number is used to store the left seats number for each row 
aircraft_left_number = pd.DataFrame(index=aircraft.index,columns = ["left_seats_number"])# setting index as aircraft
aircraft_left_number["left_seats_number"] = len(columns)

aircraft_left_index = pd.DataFrame(index=aircraft.index,columns = ["left_seats_Index"])# setting index as aircraft
aircraft_left_index["left_seats_Index"] = 0

### define functions that used in the main

In [22]:
def update_metrics_table(refused, separated):
    c.execute("UPDATE metrics SET passengers_refused = ? , passengers_separated = ? ;",(refused,separated))
    conn.commit() # this is a Python command

def small_than_max(aircraft_left_number, people_in_group, booking_name):
    best_row_index = find_the_best_row(aircraft_left_number, people_in_group)
    fit_into_aircraft(people_in_group,best_row_index, booking_name)
    
def fit_into_aircraft(people_in_group,best_row_index, booking_name):
    i = 0 
    left_seats = aircraft_left_number.iloc[best_row_index]["left_seats_number"] 
    left_index = aircraft_left_index.iloc[best_row_index]["left_seats_Index"]
    while(i < people_in_group):
        new_index = left_index + i
        if aircraft.iloc[best_row_index][new_index] == 0:
            aircraft.iloc[best_row_index][new_index] = 1
            #update to the database, since the sircraft changed
            row = best_row_index + 1
            seat = seat_letter[new_index]
            update_seating_table(booking_name, row, seat)
            
        i += 1
    aircraft_left_number.iloc[best_row_index]["left_seats_number"] = left_seats - people_in_group
    aircraft_left_index.iloc[best_row_index]["left_seats_Index"] = left_index + people_in_group

    
    
def update_seating_table(booking_name, row, seat):
    query = ("UPDATE seating SET name = '%s' WHERE  row = %d AND seat = '%s';" % (booking_name, row, seat))
    c.execute(query)
    conn.commit()

    
#this function is used for find the best number of seat to fit in all people in the booking group
#return the best row index
#the next function should be update aircraft 
def find_the_best_row(aircraft_left_number, people_in_group):
    numpyMatrix = aircraft_left_number["left_seats_number"].as_matrix()
    best_row_index = find_nearest_above(numpyMatrix, people_in_group)

    return best_row_index
    
def find_nearest_above(my_array, target):
    diff = my_array - target
    mask = np.ma.less(diff, 0)
    # We need to mask the negative differences and zero
    # since we are looking for values above
    if np.all(mask):
        return None # returns None if target is greater than any value
    masked_diff = np.ma.masked_array(diff, mask)
    return masked_diff.argmin()

    
def split_into_two_group(people_in_group):
    if(people_in_group%2 == 1):
        people_in_group_1 = (people_in_group - 1)/2
        people_in_group_2 = (people_in_group + 1)/2
        same = 0
    else:
        people_in_group_1 = people_in_group_2 = people_in_group/2
        same = 1
    return same, people_in_group_1, people_in_group_2

# new one:
def bigger_than_max(aircraft_left_number,people_in_group, split_number):
    result = split_number
    left_max = aircraft_left_number["left_seats_number"].max()
    same, people_in_group_1, people_in_group_2 = split_into_two_group(people_in_group)
    if people_in_group_1 == 1:
        split_number += 1
        result = split_number
    if people_in_group_2 == 1:
        split_number += 1
        result = split_number
    #same = 1
    if same == 1:
        if (people_in_group_1 > left_max) & (people_in_group_2 > left_max) :
            bigger_than_max(aircraft_left_number,people_in_group_1, split_number)
            bigger_than_max(aircraft_left_number,people_in_group_2, split_number)
        else:
            small_than_max(aircraft_left_number, people_in_group_1, booking_name)
            if people_in_group_2 <= aircraft_left_number["left_seats_number"].max(): 
                small_than_max(aircraft_left_number, people_in_group_2, booking_name)
            else:
                bigger_than_max(aircraft_left_number,people_in_group_2, split_number)
    else:
        if (people_in_group_1 > left_max) & (people_in_group_2 > left_max) :
            bigger_than_max(aircraft_left_number,people_in_group_1, split_number)
            bigger_than_max(aircraft_left_number,people_in_group_2, split_number)
        else:
            if (people_in_group_1 <= left_max) & (people_in_group_2 <= left_max) :
                small_than_max(aircraft_left_number, people_in_group_2, booking_name)
                if people_in_group_1 <= aircraft_left_number["left_seats_number"].max():
                    small_than_max(aircraft_left_number, people_in_group_1, booking_name)
                else: 
                    bigger_than_max(aircraft_left_number,people_in_group_1, split_number)
            else:
                small_than_max(aircraft_left_number, people_in_group_1, booking_name)
                bigger_than_max(aircraft_left_number,people_in_group_2, split_number)
    
    return result

### main function to locate seats on the airplane

In [23]:
i = 0
refused = 0
separated = 0
seats_located = 0
# total_bookings_number = 3
while(i < total_bookings_number):
    people_in_group = bookings.iloc[i][1]
    booking_name = bookings.iloc[i][0]
    seats_located += people_in_group
    max_left_seats_row = aircraft_left_number["left_seats_number"].max()
    
    
    if seats_located > seats_number: #only refuse when do not have enough seats
        refused += 1                  #otherwise located sepratedly 
        seats_located -= people_in_group
        #update refused and sparated to the database
        update_metrics_table(refused, separated)
        
    elif seats_located == seats_number: 
        # do the same as seats_located < seats_number:
        if people_in_group <= max_left_seats_row:
            small_than_max(aircraft_left_number, people_in_group, booking_name)
        else:
            separated = bigger_than_max(aircraft_left_number,people_in_group, separated)
            
        refused += (total_bookings_number - i)
        #update refused and sparated to the database
        update_metrics_table(refused, separated)
        break
    else:
        #seats_located < seats_number
        #locate seats
        if people_in_group <= max_left_seats_row:
            small_than_max(aircraft_left_number, people_in_group, booking_name)
            pd.read_sql_query("SELECT * FROM seating;", conn) # give the table name and the connection

        else:
            separated = bigger_than_max(aircraft_left_number,people_in_group, separated)
            #update to db(refuse_number, seat_away_number)
            update_metrics_table(refused, separated)
    
    
    i += 1
 

### check updated database

In [24]:
aircraft_df = pd.read_sql_query("SELECT * FROM seating;", conn) # give the table name and the connection
aircraft_df

,row,seat,name
0,1,A,Janis Mccullough
1,2,A,Maria Bonine
2,3,A,Jesse Stacy
3,4,A,Louis Cozzolino
4,5,A,Roy Scott
5,6,A,Doris Banks
6,7,A,Hilda Murphy
7,8,A,Kenneth Hudson
8,9,A,Amy Bernard
9,10,A,James Wilson
